In [1]:
import findspark
findspark.init()
from pyspark import SparkContext
from pyspark.sql import *
from pyspark import SparkConf
import pyspark as ps
import pandas as pd

In [2]:
conf = (SparkConf()
            .setAppName("TestRdd") \
            .set('spark.driver.cores', '4') \
            .set('spark.executor.cores', '4') \
            .set('spark.driver.memory', '32G') \
            .set('spark.executor.memory', '32G'))\
            .set("spark.sql.broadcastTimeout", "72000")\
            .set('spark.shuffle.service.enabled', 'TRUE') \
            .set("spark.debug.maxToStringFields" , "100")\
            .set("spark.executor.heartbeatInterval", "14400")\
            .set("spark.network.timeout", "70000s")
sc = SparkContext('local', conf = conf)
sqlContext = SQLContext(sc)

In [3]:
data_reviews =  sqlContext.read.parquet('C:/Users/tomas/OneDrive/Escritorio/ORGA DE DATOS/review.parquet')
data_tips = sqlContext.read.parquet('C:/Users/tomas/OneDrive/Escritorio/ORGA DE DATOS/tip.parquet')
data_negocios = sqlContext.read.parquet('C:/Users/tomas/OneDrive/Escritorio/ORGA DE DATOS/business.parquet')

In [4]:
reviews = data_reviews.rdd
tips = data_tips.rdd
negocios = data_negocios.rdd

In [5]:
tips_fuck = tips.filter(lambda x:  x.text is not None and "fuck" in x.text).map(lambda x: ((x.business_id,x.user_id),x.text.count("fuck")))
reviews_fuck = reviews.filter(lambda x: "fuck" in x.text).map(lambda x: ((x.business_id,x.user_id),(x.text.count("fuck"),x.stars)))

In [12]:
#Para las reviews, ¿cuál es el promedio del score?

In [13]:
reviews_fuck.map(lambda x: x[1][1]).reduce(lambda x,y: x+y)/reviews_fuck.count()
#Mapeo (x.text.count(fuck)) cuenta los "fuck" en el texto

1.9745693191140279

In [ ]:
#¿Cuál es el usuario que más veces usó la palabra en ambas tablas?

In [22]:
usuarios_reviews_fuck = reviews_fuck.map(lambda x: (x[0][1],x[1][0])).reduceByKey(lambda x,y: x+y)
usuarios_tips_fuck = tips_fuck.map(lambda x: (x[0][1],x[1])).reduceByKey(lambda x,y: x+y)

In [26]:
usuarios_reviews_fuck.fullOuterJoin(usuarios_tips_fuck).map(lambda x: (x[0],(x[1][0] or 0)+(x[1][1] or 0)))\
                     .reduce(lambda x,y: x if x[1] > y[1] else y)

('oR5az_eNCnfN7e49H3ONhg', 280)

In [12]:
#¿Cuál es el negocio que más veces recibió la palabra en ambas tablas?

In [18]:
negocios_reviews_fuck = reviews_fuck.map(lambda x: (x[0][0],x[1][0])).reduceByKey(lambda x,y: x+y)
negocios_tips_fuck = tips_fuck.map(lambda x: (x[0][0],x[1])).reduceByKey(lambda x,y: x+y)

In [21]:
negocios_reviews_fuck.fullOuterJoin(negocios_tips_fuck).map(lambda x: (x[0],(x[1][0] or 0)+(x[1][1] or 0)))\
                     .reduce(lambda x,y: x if x[1] > y[1] else y)

('BnzAXu_3S2CRQOG_vMIYRw', 14)

In [16]:
#¿Cuál es el negocio de más de 3 reviews con mayor ratio de textos que contienen “fuck”?

In [10]:
cant_reviews = reviews_fuck.count()

In [6]:
negocios_reviews = negocios.filter(lambda x: x.review_count > 3).map(lambda x: (x.business_id,int(x.review_count)))

In [7]:
ratio_negocios_fuck = negocios_reviews.join(reviews_fuck.map(lambda x: (x[0][0],x[1][0])).reduceByKey(lambda x,y: x+y))

In [11]:
ratio_negocios_fuck.map(lambda x: (x[0],x[1][1]/cant_reviews)).reduce(lambda x,y: x if x[1] > y[1] else y) 

('BnzAXu_3S2CRQOG_vMIYRw', 0.0022969647251845776)

In [57]:
#Para los negocios con textos que tienen “fuck”
#correlacione el rating promedio del negocio con su ratio de “fuck” en textos.

In [16]:
negocios_rating_reviews = negocios.filter(lambda x: x.review_count != 0)\
                                  .map(lambda x: (x.business_id,(int(x.stars),int(x.review_count))))

In [6]:
r = reviews_fuck.map(lambda x: (x[0][0],x[1][0])).reduceByKey(lambda x,y: x+y)
t = tips_fuck.map(lambda x: (x[0][0],x[1])).reduceByKey(lambda x,y: x+y)

In [15]:
reviews_tips_fuck = r.fullOuterJoin(t).map(lambda x: (x[0],(x[1][0] or 0)+(x[1][1] or 0)))

In [12]:
negocios_reviews_tips = negocios_rating_reviews.join(rt)

In [13]:
correlacion = negocios_reviews_tips.map(lambda x: (x[1][0][0],x[1][1]/x[1][0][1]))

In [14]:
from pyspark.mllib.stat import Statistics
print(Statistics.corr(correlacion, method="pearson")) #CORRELACION

[[ 1.         -0.22558112]
 [-0.22558112  1.        ]]
